In [11]:
import os
import sys
import pandas as pd
import numpy as np
import pywt
import scipy.io as spio
from scipy.stats import kurtosis as kurto
from scipy.stats import skew as skewness
from scipy.stats import entropy
from collections import Counter
from sklearn import svm
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

import timeit
import warnings
warnings.filterwarnings("ignore")

In [12]:
# reading the channel order for dataframe
channel_order = pd.read_excel("channel-order.xlsx", header=None)

**Feature extraction**

In [13]:
def get_skew(data_set):
    return skewness(data_set)

In [14]:
def get_kurtosis(data_set):
    return kurto(data_set)

In [15]:
def get_mean(data_set):
    return np.mean(data_set)

In [16]:
def get_wavelet_energy(data_set):
    wavelet_energy = np.nansum(np.log2(np.square(data_set)))
    return round(wavelet_energy, 3)

In [17]:
def get_shannon_entropy(data_set):
    data_set = np.square(data_set)
    data_set = data_set/np.sum(data_set)
    shannon_entropy = -np.nansum(data_set * np.log2(data_set))
    return round(shannon_entropy, 3)

##### Training

In [ ]:
for MAX_LEVEL in range(5,11):
    for WAVELET in ["db10","db9","db8","db7","db6","db5","db4","db3","db2","db1"]:
        num_features = 5
        final_stored_features = pd.DataFrame(columns = range(int(MAX_LEVEL*62*num_features)))
        for file_name in os.listdir("Preprocessed_EEG/")[:-2]: #45 file_names (3*trials, 15 participants)
            eeg_recording = spio.loadmat("Preprocessed_EEG/" + file_name)
            useful_keys = [key for key, values in eeg_recording.items() if key != '__header__' and key != '__version__' and key != '__globals__' ]

            for data_file_name in useful_keys: #15 arrays for 15 movies
                movie_clip_dataframe = pd.DataFrame(eeg_recording[data_file_name])
                M,N = movie_clip_dataframe.shape[0], movie_clip_dataframe.shape[1]
                # Feature extraction part of the module
                entropy = []
                energy = []
                mean = []
                kurtosis = []
                skew = []
                for channel in movie_clip_dataframe.iterrows():  # Iterating through the 62 channels
                    dwt_bands = []
                    approx_coeff = channel[1]
                    #Features used
                    temp_ent = []
                    temp_eng = []
                    temp_mean = []
                    temp_kurtosis = []
                    temp_skew = []

                    for i in range(MAX_LEVEL):  # number of levels levels
                        (approx_coeff, detail_coeff) = pywt.dwt(approx_coeff, WAVELET)
                        dwt_bands.append(detail_coeff)
                    n = len(dwt_bands)
                    for i in range(n):  # DWT_bands = 23504, 11755
                        temp_ent.append(get_shannon_entropy(dwt_bands[n - i - 1])) #scalar value
                        temp_eng.append(get_wavelet_energy(dwt_bands[n - i - 1]))  #scalar value
                        temp_mean.append(get_mean(dwt_bands[n - i - 1]))
                        temp_kurtosis.append(get_kurtosis(dwt_bands[n - i - 1]))
                        temp_skew.append(get_skew(dwt_bands[n - i - 1]))
                    #Features used
                    entropy.append(temp_ent)
                    energy.append(temp_eng)
                    mean.append(temp_mean)
                    kurtosis.append(temp_kurtosis)
                    skew.append(temp_skew)

                flattened_entropy = []
                flattened_energy = []
                flattened_mean = []
                flattened_kurtosis = []
                flattened_skew = []

                for i in range(len(entropy)):
                    for j in range(len(entropy[0])):
                        flattened_entropy.append(entropy[i][j])
                        flattened_energy.append(energy[i][j])
                        flattened_mean.append(mean[i][j])
                        flattened_kurtosis.append(kurtosis[i][j])
                        flattened_skew.append(skew[i][j])
                #concatenate the features
                features = flattened_entropy + flattened_energy + flattened_mean + flattened_kurtosis + flattened_skew
                final_stored_features.loc[len(final_stored_features.index)] = features
            print(file_name)
        #PCA
        normalised_features = pd.DataFrame(normalize(final_stored_features, axis = 0))
        covariance_matrix = normalised_features.cov()
        U, sigma, VT = np.linalg.svd(covariance_matrix)
        projected_PCA_data = normalised_features@U[:,:361]
        
        labels = pd.read_csv("outputs_main.csv", header=None)
        X = projected_PCA_data.iloc[:, :].values
        Y = labels.iloc[:, :].values
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=42)
        #Model Training and grid search
        svc = SVC()
        parameters = {"C": (100, 1e3, 1e4, 1e5, 1e6, 1e7, 1e8, 1e9), "gamma": (1e-08, 1e-7, 1e-6, 1e-5),"kernel":("linear", "rbf","poly")}
        grid_search = GridSearchCV(svc, parameters, n_jobs=-1, cv=5)
        start_time = timeit.default_timer()
        grid_search.fit(X_train, Y_train)
        print(grid_search.best_params_)
        svc_best = grid_search.best_estimator_
        accuracy = svc_best.score(X_test, Y_test)
        accuracy_train = svc_best.score(X_train, Y_train)
        print("Accuracy on the training set is: {0:.1f}%".format(accuracy_train*100))
        print("Accuracy on the testing set is: {0:.1f}%".format(accuracy*100))
        prediction = svc_best.predict(X_test)
        report = classification_report(Y_test, prediction)
        print('RESULTS FOR WAVELET: ',WAVELET)
        print('SVM',report)

        gnb = GaussianNB()
        gnb.fit(X_train, Y_train)
        accuracy = gnb.score(X_test, Y_test)
        print("Accuracy on the testing set is: {0:.1f}%".format(accuracy*100))
        prediction = gnb.predict(X_test)
        report = classification_report(Y_test, prediction)
        print('GaussianNB',report)

        rf = RandomForestClassifier()
        parameters = {"n_estimators": (10,20,30,50,70,100), 'max_depth':(1,2,3,4)}
        grid_search = GridSearchCV(rf, parameters, n_jobs=-1, cv=5)
        grid_search.fit(X_train, Y_train)
        print(grid_search.best_params_)
        rf_best = grid_search.best_estimator_
        accuracy = rf_best.score(X_test, Y_test)
        accuracy_train = rf_best.score(X_train, Y_train)
        print("Accuracy on the training set is: {0:.1f}%".format(accuracy_train*100))
        print("Accuracy on the testing set is: {0:.1f}%".format(accuracy*100))
        prediction = rf_best.predict(X_test)
        report = classification_report(Y_test, prediction)
        print('RandomForest',report)

        knn = KNeighborsClassifier()
        parameters = {'n_neighbors':range(1,21)};
        grid_search = GridSearchCV(knn, parameters, n_jobs=-1, cv=5)
        grid_search.fit(X_train, Y_train)
        print(grid_search.best_params_)
        knn_best = grid_search.best_estimator_
        accuracy = knn_best.score(X_test, Y_test)
        accuracy_train = knn_best.score(X_train, Y_train)
        print("Accuracy on the training set is: {0:.1f}%".format(accuracy_train*100))
        print("Accuracy on the testing set is: {0:.1f}%".format(accuracy*100))
        prediction = knn_best.predict(X_test)
        report = classification_report(Y_test, prediction)
        print('KNN',report)


10_20131130.mat
10_20131204.mat
10_20131211.mat
11_20140618.mat
11_20140625.mat
11_20140630.mat
12_20131127.mat
12_20131201.mat
12_20131207.mat
13_20140527.mat
13_20140603.mat
13_20140610.mat
14_20140601.mat
14_20140615.mat
14_20140627.mat
15_20130709.mat
15_20131016.mat
15_20131105.mat
1_20131027.mat
1_20131030.mat
1_20131107.mat
2_20140404.mat
2_20140413.mat
2_20140419.mat
3_20140603.mat
3_20140611.mat
3_20140629.mat
4_20140621.mat
4_20140702.mat
4_20140705.mat
5_20140411.mat
5_20140418.mat
5_20140506.mat
6_20130712.mat
6_20131016.mat
6_20131113.mat
7_20131027.mat
7_20131030.mat
7_20131106.mat
8_20140511.mat
8_20140514.mat
8_20140521.mat
9_20140620.mat
9_20140627.mat
9_20140704.mat
{'C': 100000000.0, 'gamma': 1e-05, 'kernel': 'rbf'}
Accuracy on the training set is: 87.8%
Accuracy on the testing set is: 57.0%
RESULTS FOR WAVELET:  db10
SVM               precision    recall  f1-score   support

          -1       0.51      0.53      0.52        47
           0       0.49      0.50     